# Foreign Whispers


## Milestone 1: Source Videos and Closed Captions


In [1]:
from pytube import YouTube, Playlist
import os
from moviepy.editor import VideoFileClip

import whisper

In [2]:
# Ensure the videos directory exists
os.makedirs('videos', exist_ok=True)

# List of URLs from the 60 Minutes Interviews playlist on YouTube.
video_urls = [
    'https://www.youtube.com/watch?v=G3Eup4mfJdA&list=PLI1yx5Z0Lrv77D_g1tvF9u3FVqnrNbCRL&index=1',
    # 'https://www.youtube.com/watch?v=480OGItLZNo&list=PLI1yx5Z0Lrv77D_g1tvF9u3FVqnrNbCRL&index=2',
    # 'https://www.youtube.com/watch?v=ervLwxz7xPo&list=PLI1yx5Z0Lrv77D_g1tvF9u3FVqnrNbCRL&index=11',
    # 'https://www.youtube.com/watch?v=OA2Tj75T3fI&list=PLI1yx5Z0Lrv77D_g1tvF9u3FVqnrNbCRL&index=4',
    # 'https://www.youtube.com/watch?v=qrvK_KuIeJk&list=PLI1yx5Z0Lrv77D_g1tvF9u3FVqnrNbCRL&index=5',
    # 'https://www.youtube.com/watch?v=oFVuQ0RP_As&list=PLI1yx5Z0Lrv77D_g1tvF9u3FVqnrNbCRL&index=6',
    # 'https://www.youtube.com/watch?v=4aPp8KX6EiU&list=PLI1yx5Z0Lrv77D_g1tvF9u3FVqnrNbCRL&index=7',
    # 'https://www.youtube.com/watch?v=h8PSWeRLGXs&list=PLI1yx5Z0Lrv77D_g1tvF9u3FVqnrNbCRL&index=8',
    # 'https://www.youtube.com/watch?v=Z8qC2tVkGeU&list=PLI1yx5Z0Lrv77D_g1tvF9u3FVqnrNbCRL&index=9',
    # 'https://www.youtube.com/watch?v=Y9nM_9oBj2k&list=PLI1yx5Z0Lrv77D_g1tvF9u3FVqnrNbCRL&index=10',

]

for url in video_urls:
    try:
        # Create a YouTube object with the URL.
        yt = YouTube(url)
        
        # Download the best quality video
        video_stream = yt.streams.get_highest_resolution()
        if video_stream:
            video_stream.download(output_path='videos/')
            print(f'Downloaded video: {yt.title}.mp4')
        
        # Download the English captions (subtitles)
        en_caption = yt.captions.get_by_language_code('en')
        if en_caption:
            en_caption_srt = en_caption.generate_srt_captions()
            with open(f'videos/{yt.title}.srt', 'w') as file:
                file.write(en_caption_srt)
                print(f'Downloaded caption: {yt.title}.srt')
    except Exception as e:
        print(f'Failed to download {url}: {str(e)}')


Downloaded video: Vice President Kamala Harris: The 2023 60 Minutes Interview.mp4


In [38]:
#save model to file in the h5 format
model.save('model.h5')


AttributeError: 'Whisper' object has no attribute 'save'

## Milestone 2: Speech to Text


In [13]:
# You may need to install whisper if you haven't already:
# !pip install openai-whisper

import os
import subprocess
import whisper

# Initialize the Whisper model
model = whisper.load_model("base")  # You can choose different models like "small", "medium", "large", or "base".

def extract_audio(video_path, audio_output_path):
    # Command to extract audio using ffmpeg
    command = f"ffmpeg -i \"{video_path}\" -ab 160k -ac 2 -ar 44100 -vn \"{audio_output_path}\""
    try:
        output = subprocess.check_output(command, stderr=subprocess.STDOUT, shell=True)
        print(output.decode('utf-8'))
    except subprocess.CalledProcessError as e:
        print(f"Error occurred: {e.output.decode('utf-8')}")

def transcribe_audio(audio_path):
    # Load audio and run the Whisper model to transcribe it
    result = model.transcribe(audio_path)
    return result['text']

# Directory where the videos are saved
video_dir = 'videos'
audio_dir = 'audio'
os.makedirs(audio_dir, exist_ok=True)

# Iterate through each downloaded video and process it
for video_file in os.listdir(video_dir):
    if video_file.endswith(".mp4"):
        video_path = os.path.join(video_dir, video_file)
        print(f"Processing {video_path}")
        audio_output_path = os.path.join(audio_dir, os.path.splitext(video_file)[0] + '.mp3')
        
        # Extract the audio from the video
        #extract_audio(video_path, audio_output_path)
        
        # Transcribe the audio to text
        text = transcribe_audio(audio_output_path)
        
        # Save the transcription to a file
        text_file_path = os.path.join(audio_dir, os.path.splitext(video_file)[0] + '.txt')
        with open(text_file_path, 'w') as text_file:
            text_file.write(text)
        print(f"Transcribed audio from {video_file} to {text_file_path}")


Processing videos/Vice President Kamala Harris The 2023 60 Minutes Interview.mp4
Transcribed audio from Vice President Kamala Harris The 2023 60 Minutes Interview.mp4 to audio/Vice President Kamala Harris The 2023 60 Minutes Interview.txt


# Milestone 3:Source Text to Target Text (25 points)

In [24]:
import nltk

nltk.download('punkt')  # Make sure to download the punkt tokenizer data


[nltk_data] Downloading package punkt to /Users/apple/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [29]:
from googletrans import Translator

def translate_text(text, target_language):
    translator = Translator()
    #slice all text on the basis of sentences and translate each sentence
    text = nltk.sent_tokenize(text)
    translation = ""
    for i in text:
        translation = translation + translator.translate(i, dest=target_language).text + "."
    return translation

translated_dir = 'translated'
os.makedirs(translated_dir, exist_ok=True)
# Iterate through each transcribed text file and translate it
for text_file in os.listdir(audio_dir):
    if text_file.endswith(".txt"):
        text_file_path = os.path.join(audio_dir, text_file)
        
        # Read the transcribed text
        with open(text_file_path, 'r') as file:
            original_text = file.read()
        
        # Translate the text to the desired language
        target_language = "es"  # Replace with your target language code
        translated_text = translate_text(original_text, target_language)
        
        # Save the translated text to a new file
        translated_text_file_path = os.path.join(translated_dir, os.path.splitext(text_file)[0] + f'.txt')
        with open(translated_text_file_path, 'w') as translated_text_file:
            translated_text_file.write(translated_text)
        print(f"Translated {text_file} to {translated_text_file_path}")

Translated Vice President Kamala Harris The 2023 60 Minutes Interview.txt to translated/Vice President Kamala Harris The 2023 60 Minutes Interview.txt


# Milestone 4: Target Text to Speech (25 points)

In [31]:
!pip install gtts

In [32]:
from gtts import gTTS

In [33]:
translated_audio_dir = 'translated_audio'
os.makedirs(translated_audio_dir, exist_ok=True)
def text_to_speech(translated_text, target_language):
    # Convert translated text to speech using gTTS
    tts = gTTS(text=translated_text, lang=target_language, slow=False)
    return tts

    # Use the generated audio file in your application logic

# Iterate through each translated text file and convert it to speech
for text_file in os.listdir(translated_dir):
    if text_file.endswith(".txt"):
        text_file_path = os.path.join(translated_dir, text_file)
        
        # Read the translated text
        with open(text_file_path, 'r') as file:
            translated_text = file.read()
        
        # Convert the translated text to speech
        target_language = "es"  # Replace with your target language code
        tts = text_to_speech(translated_text, target_language)
        
        # Save the translated text to a new file
        translated_audio_file_path = os.path.join(translated_audio_dir, os.path.splitext(text_file)[0] + f'.mp3')
        tts.save(translated_audio_file_path)
        print(f"Converted {text_file} to {translated_audio_file_path}")

Converted Vice President Kamala Harris The 2023 60 Minutes Interview.txt to translated_audio/Vice President Kamala Harris The 2023 60 Minutes Interview.mp3
